## Process the image data 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

import skimage
from skimage import io, measure, exposure
from skimage.color import rgba2rgb, rgb2gray
from skimage.feature import greycomatrix, greycoprops

from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report


import os
import csv

%matplotlib inline

In [2]:
print(skimage.__version__)

0.17.2


In [3]:
def extract_features(image_input):
    gray =  rgb2gray(image_input)
    v1 = image_input.flatten()
    
    v = np.concatenate([v1])
    
    return v

In [4]:
directory = "Border_aug/"

border_features = []

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        image = io.imread(directory + filename)
        v = extract_features(image)
        v_list = [filename, 0] + v.tolist()
        border_features.append(v_list)

In [5]:
directory = "Center_aug/"

center_features = []

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        image = io.imread(directory + filename)
        v = extract_features(image)
        v_list = [filename, 1] + v.tolist()
        center_features.append(v_list)

In [7]:
directory = "Blank_aug/"
blank_features = []

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        image = io.imread(directory + filename)
        v = extract_features(image)
        v_list = [filename, 2] + v.tolist()
        blank_features.append(v_list)

In [8]:
features = []
labels = []

for row in border_features:
    filename = row[0]
    class_label = row[1]
    v = np.array(row[2:len(row)], dtype=np.float32)
    f = [filename, class_label, v]
    features.append(f)
    labels.append(class_label)

In [9]:
for row in center_features:
    filename = row[0]
    class_label = row[1]
    v = np.array(row[2:len(row)], dtype=np.float32)
    f = [filename, class_label, v]
    features.append(f)
    labels.append(class_label)

In [10]:
for row in blank_features:
    filename = row[0]
    class_label = row[1]
    v = np.array(row[2:len(row)], dtype=np.float32)
    f = [filename, class_label, v]
    features.append(f)
    labels.append(class_label)

### Step 2: Split the data into training and testing

In [11]:
feature_length = len(features[0][2])
features, labels = shuffle(features, labels, random_state=0)

In [12]:
N_train = 60
features_train = features[0:N_train]
features_test = features[N_train:len(features)]

labels_train = labels[0:N_train]
labels_test = labels[N_train:len(features)]

In [13]:
data_train = np.zeros((len(features_train), feature_length))
data_test  = np.zeros((len(features_test), feature_length))

In [14]:
for i in range(0, len(features_train)):
    data_train[i, :] = features_train[i][2]
    
for i in range(0, len(features_test)):
    data_test[i, :] = features_test[i][2]

### Step 3: Set up the classifier and train it


In [38]:
clf = SVC(random_state=0, max_iter=500,kernel = "rbf")
#"linear","poly","rbf","sigmoid"

In [39]:
clf.fit(data_train, np.asarray(labels_train))

SVC(max_iter=500, random_state=0)

In [40]:
prediction = clf.predict(data_test)

In [41]:
print(classification_report(labels_test, prediction))

              precision    recall  f1-score   support

           0       0.70      0.90      0.79       470
           1       0.84      0.75      0.79       488
           2       0.94      0.78      0.85       479

    accuracy                           0.81      1437
   macro avg       0.83      0.81      0.81      1437
weighted avg       0.83      0.81      0.81      1437

